In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
IMAGE_SIZE = 216

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout , BatchNormalization

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5500)])
  except RuntimeError as e:
    print(e)

In [ ]:
Dataset_Path = os.getcwd() + '/inaturalist_12K'
labels = ["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
DATSET_SIZE = 10000

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import cv2
import csv
import PIL


# Dataset_Path = '/content/drive/MyDrive/inaturalist_12K'
train_path = os.path.join(Dataset_Path , 'train')
test_path = os.path.join(Dataset_Path , 'val')
# test_path1 = test_path
test_path1 = 'D:/val'

In [ ]:
import pathlib
Dataset_Path = pathlib.Path(Dataset_Path)
train_path = pathlib.Path(train_path)
test_path = pathlib.Path(test_path)

In [ ]:
Mammals = list(train_path.glob('Mammalia/*'))
PIL.Image.open(str(Mammals[0]))

In [ ]:
# Preprocessing

In [ ]:
batch_size = 64

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="training",
    interpolation='bilinear'
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="validation",
    interpolation='bilinear'
)

In [ ]:
type(train_ds)
type(val_ds)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras.layers import experimental

In [ ]:
## Wandb sweep

In [ ]:
#### Install and login

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login(key="866040d7d81f67025d43e7d50ecd83d54b6cf977", relogin=False)

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [16,32]
        },
        'num_fliters': {  
            'values': [16,48]
        },
        'filter_size':{
            'values': [3]
        },
        'filter_org': {
            'values': ['same']
        },
        'data_aug': {
            'values': ['yes']
        },
        'dropout':{
            'values': [0.01,0.1]
        },
        'batch_norm':{
            'values' : ['yes']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="mooizz",project="conv_inaturalist")

In [ ]:
data_augmentation = Sequential(
  [
    experimental.preprocessing.RandomFlip("horizontal_and_vertical", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    experimental.preprocessing.RandomRotation(0.3),
    experimental.preprocessing.RandomZoom(0.2),
  ]
)

In [ ]:
def getModel(numFilters, filterSize,filterConf,dropout,data_aug, batch_norm):
    model = Sequential()
    if data_aug == 'yes':
        model.add(data_augmentation)
    input_shape1 = (IMAGE_SIZE, IMAGE_SIZE, 3)
    model.add(experimental.preprocessing.Rescaling(1./255, input_shape=input_shape1))
    model.add(Conv2D(numFilters, (filterSize, filterSize), input_shape=input_shape1))
    if batch_norm == 'yes':
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))        

    for l in range(4):
        if filterConf == 'double':
            numFilters = numFilters * 2
        if filterConf == 'half':
            numFilters = numFilters / 2
        model.add(Conv2D(numFilters, (filterSize, filterSize)))
        if batch_norm == 'yes':
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model

In [ ]:
def train():
    config_defaults = {
        'epochs' : 22,
        'batch_size' : 64,
        'optimizer' : 'adam',
        'filter_org' : 'same',
        'num_filters' : 64,
        'filter_size' : 5,
        'dropout' : 0.2,
        'data_aug' : 'yes',
         'batch_norm' : 'yes'
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    model = getModel(config.num_filters,config.filter_size,config.filter_org,
                     config.dropout, config.data_aug, config.batch_norm)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
    model.fit(train_ds,
          epochs=config.epochs,
          validation_data=val_ds,
          verbose=1,
             callbacks=[WandbCallback()])
   

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()